# Resumen general

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import cohen_kappa_score

In [2]:
anomalias = ['Anomalia','Negro','Congelado','Hum','CurvaCola']
metricas = ['roc_auc','accuracy','precision','kappa','sensibilidad','especificidad']

def calc_metricas(metodos,metodos_lista):
    metodos_metricas = pd.DataFrame(index=metodos_lista,columns=metricas)
    for metodo in metodos_lista:
        metodos_metricas.loc[metodo,metricas[0]] = roc_auc_score(metodos['Anomalia'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[1]] = accuracy_score(metodos['Anomalia'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[2]] = precision_score(metodos['Anomalia'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[3]] = cohen_kappa_score(metodos['Anomalia'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[4]] = recall_score(metodos['Anomalia'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[5]] = recall_score(metodos['Anomalia'], metodos[metodo], pos_label=0)
    return metodos_metricas

def calc_sensibilidad(metodos,metodos_lista,anomalias):
    metodos_sensibilidades = pd.DataFrame(index=metodos_lista, columns=anomalias)
    for metodo in metodos_lista:
        for anomalia in anomalias:
            metodos_sensibilidades.loc[metodo,anomalia] = recall_score(metodos[anomalia], metodos[metodo])
    return metodos_sensibilidades

## Métodos

In [3]:
metodos = pd.read_csv('Metodos/PyOD/Resultados/HBOS_completo.csv',index_col='fecha')
metodos = metodos.drop('Y_pred', axis=1)

metodos['HBOS'] = pd.read_csv('Metodos/PyOD/Resultados/HBOS_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['IForest'] = pd.read_csv('Metodos/PyOD/Resultados/IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['KNN'] = pd.read_csv('Metodos/PyOD/Resultados/KNN_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['LODA'] = pd.read_csv('Metodos/PyOD/Resultados/LODA_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MAD_PyOD'] = pd.read_csv('Metodos/PyOD/Resultados/MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MCD'] = pd.read_csv('Metodos/PyOD/Resultados/MCD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos['IForestASD'] = pd.read_csv('Metodos/PySAD/Resultados/IForestASD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MAD_PySAD'] = pd.read_csv('Metodos/PySAD/Resultados/MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['SAD'] = pd.read_csv('Metodos/PySAD/Resultados/SAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['xStream'] = pd.read_csv('Metodos/PySAD/Resultados/xStream_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_htm = pd.read_csv('Metodos/HTMs/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_htm['HTM'] = metodos_htm['Y_pred']
metodos_htm = metodos_htm.drop('Y_pred',axis=1)

metodos.head()

,Negro,CurvaCola,Congelado,Hum,Anomalia,HBOS,IForest,KNN,LODA,MAD_PyOD,MCD,IForestASD,MAD_PySAD,SAD,xStream
fecha,,,,,,,,,,,,,,,
2021-08-01 11:06:00,0,1,0,0,1,1,1,1,0,0,1,1,0,0,1
2021-03-15 11:15:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-10-19 07:04:00,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
2021-04-17 10:46:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-03-07 19:50:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Métricas generales:

In [4]:
metodos_lista = ['HBOS','IForest','KNN','LODA','MAD_PyOD','MCD', 'IForestASD', 'MAD_PySAD','SAD','xStream']
metodos_metricas = calc_metricas(metodos,metodos_lista)
metodos_metricas = pd.concat([metodos_metricas,calc_metricas(metodos_htm, ['HTM'])])
metodos_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
HBOS,0.537511,0.670005,0.493402,0.091176,0.150848,0.924173
IForest,0.5631,0.693173,0.610557,0.153622,0.183503,0.942696
KNN,0.520688,0.654686,0.418294,0.050221,0.129641,0.911736
LODA,0.55061,0.682082,0.554308,0.123191,0.16693,0.934289
MAD_PyOD,0.59333,0.729708,0.916837,0.233569,0.195333,0.991326
MCD,0.530013,0.663358,0.46036,0.072947,0.140871,0.919156
IForestASD,0.581826,0.673911,0.506348,0.181312,0.313089,0.850562
MAD_PySAD,0.587499,0.728848,1.0,0.221671,0.174999,1.0
SAD,0.587499,0.728848,1.0,0.221671,0.174999,1.0
xStream,0.604199,0.687045,0.535309,0.227118,0.362427,0.845971


In [5]:
px.line(metodos_metricas.T)

### Sensibilidad por tipo de anomalía

In [6]:
metodos_sensibilidades = calc_sensibilidad(metodos,metodos_lista,anomalias)
metodos_sensibilidades = pd.concat([metodos_sensibilidades,calc_sensibilidad(metodos_htm, ['HTM'], anomalias)])
metodos_sensibilidades

,Anomalia,Negro,Congelado,Hum,CurvaCola
HBOS,0.150848,0.27432,0.293006,0.866221,0.122463
IForest,0.183503,0.431151,0.485618,0.83612,0.134419
KNN,0.129641,0.174077,0.163001,0.381271,0.122787
LODA,0.16693,0.371328,0.398195,0.842809,0.125307
MAD_PyOD,0.195333,0.737807,0.911732,1.0,0.06372
MCD,0.140871,0.216114,0.119571,0.274247,0.142691
IForestASD,0.313089,0.670978,0.701072,0.939799,0.246543
MAD_PySAD,0.174999,0.731878,0.904681,0.006689,0.057839
SAD,0.174999,0.731878,0.904681,0.006689,0.057839
xStream,0.362427,0.839935,0.897067,0.906355,0.266318


In [7]:
px.line(metodos_sensibilidades.T)

## Métodos NIRvsSecadero

In [8]:
metodos_NvS = pd.read_csv('Metodos_NIRvsSecadero/Total/Resultados/PyOD_IForest_completo.csv',index_col='fecha')
metodos_NvS = metodos_NvS.drop('Y_pred', axis=1)

metodos_NvS['IForest'] = pd.read_csv('Metodos_NIRvsSecadero/Total/Resultados/PyOD_IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_NvS['MAD'] = pd.read_csv('Metodos_NIRvsSecadero/Total/Resultados/PyOD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_NvS_htm = pd.read_csv('Metodos_NIRvsSecadero/Total/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_NvS_htm['HTM'] = metodos_NvS_htm['Y_pred']
metodos_NvS_htm = metodos_NvS_htm.drop('Y_pred',axis=1)

metodos_NvS.head()

,Negro,CurvaCola,Congelado,Hum,Anomalia,IForest,MAD
fecha,,,,,,,
2021-08-01 11:06:00,0,1,0,0,1,0,0
2021-03-15 11:15:00,0,0,0,0,0,0,0
2021-10-19 07:04:00,0,0,0,0,0,0,0
2021-04-17 10:46:00,0,0,0,0,0,0,0
2021-03-07 19:50:00,0,0,0,0,0,0,0


### Métricas generales:

In [9]:
metodos_NvS_lista = ['IForest','MAD']
metodos_NvS_metricas = calc_metricas(metodos_NvS,['IForest','MAD'])
metodos_NvS_metricas = pd.concat([metodos_NvS_metricas,calc_metricas(metodos_NvS_htm, ['HTM'])])
metodos_NvS_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
IForest,0.591815,0.71781,0.730455,0.223097,0.224118,0.959511
MAD,0.591454,0.727002,0.880853,0.228194,0.195879,0.987029
HTM,0.456615,0.609992,0.048738,-0.108785,0.010165,0.903066


In [10]:
px.line(metodos_NvS_metricas.T)

### Sensibilidad por tipo de anomalía

In [11]:
metodos_NvS_sensibilidades = calc_sensibilidad(metodos_NvS,metodos_NvS_lista,anomalias)
metodos_NvS_sensibilidades = pd.concat([metodos_NvS_sensibilidades,calc_sensibilidad(metodos_NvS_htm, ['HTM'], anomalias)])
metodos_NvS_sensibilidades

,Anomalia,Negro,Congelado,Hum,CurvaCola
IForest,0.224118,0.800593,0.909193,0.137124,0.113158
MAD,0.195879,0.757747,0.900169,0.076923,0.081879
HTM,0.010165,0.0,0.009393,0.01377,0.009644


In [12]:
px.line(metodos_NvS_sensibilidades.T)

## Métodos PVvsPred

In [13]:
metodos_PvP = pd.read_csv('Metodos_PVvsPred/Total/Resultados/PyOD_MAD_completo.csv',index_col='fecha')
metodos_PvP = metodos_PvP.drop('Y_pred', axis=1)

metodos_PvP['MAD_PyOD'] = pd.read_csv('Metodos_PVvsPred/Total/Resultados/PyOD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_PvP['MAD_PySAD'] = pd.read_csv('Metodos_PVvsPred/Total/Resultados/PySAD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_PvP['SAD'] = pd.read_csv('Metodos_PVvsPred/Total/Resultados/PySAD_SAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_PvP_htm = pd.read_csv('Metodos_PVvsPred/Total/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_PvP_htm['HTM'] = metodos_PvP_htm['Y_pred']
metodos_PvP_htm = metodos_PvP_htm.drop('Y_pred',axis=1)

metodos_PvP.head()

,Negro,CurvaCola,Congelado,Hum,Anomalia,MAD_PyOD,MAD_PySAD,SAD
fecha,,,,,,,,
2021-08-01 11:06:00,0,1,0,0,1,0,0,0
2021-03-15 11:15:00,0,0,0,0,0,0,0,0
2021-10-19 07:04:00,0,0,0,0,0,0,0,0
2021-04-17 10:46:00,0,0,0,0,0,0,0,0
2021-03-07 19:50:00,0,0,0,0,0,0,0,0


### Métricas generales:

In [14]:
metodos_PvP_lista = ['MAD_PyOD','MAD_PySAD','SAD']
metodos_PvP_metricas = calc_metricas(metodos_PvP,metodos_PvP_lista)
metodos_PvP_metricas = pd.concat([metodos_PvP_metricas,calc_metricas(metodos_PvP_htm, ['HTM'])])
metodos_PvP_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
MAD_PyOD,0.598938,0.72659,0.795289,0.242067,0.226408,0.971468
MAD_PySAD,0.601583,0.726984,0.780426,0.247209,0.235621,0.967545
SAD,0.601446,0.726913,0.78047,0.246909,0.235294,0.967598
HTM,0.45648,0.609791,0.048831,-0.109099,0.010219,0.90274


In [15]:
px.line(metodos_PvP_metricas.T)

### Sensibilidad por tipo de anomalía

In [16]:
metodos_PvP_sensibilidades = calc_sensibilidad(metodos_PvP,metodos_PvP_lista,anomalias)
metodos_PvP_sensibilidades = pd.concat([metodos_PvP_sensibilidades,calc_sensibilidad(metodos_PvP_htm, ['HTM'], anomalias)])
metodos_PvP_sensibilidades

,Anomalia,Negro,Congelado,Hum,CurvaCola
MAD_PyOD,0.226408,0.739693,0.904681,0.247492,0.114321
MAD_PySAD,0.235621,0.755591,0.904963,0.374582,0.122916
SAD,0.235294,0.7564,0.904963,0.351171,0.122851
HTM,0.010219,0.0,0.009563,0.014426,0.009657


In [17]:
px.line(metodos_PvP_sensibilidades.T)

## Métodos SoloHum

In [18]:
metodos_SH = pd.read_csv('Metodos_SoloHumedades/Total/Resultados/PyOD_HBOS_completo.csv',index_col='fecha')
metodos_SH = metodos_SH.drop('Y_pred', axis=1)

metodos_SH['HBOS'] = pd.read_csv('Metodos_SoloHumedades/Total/Resultados/PyOD_HBOS_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_SH['IForest'] = pd.read_csv('Metodos_SoloHumedades/Total/Resultados/PyOD_IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_SH['KNN'] = pd.read_csv('Metodos_SoloHumedades/Total/Resultados/PyOD_KNN_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_SH.head()

,Negro,CurvaCola,Congelado,Hum,Anomalia,HBOS,IForest,KNN
fecha,,,,,,,,
2021-08-01 11:06:00,0,1,0,0,1,0,0,1
2021-03-15 11:15:00,0,0,0,0,0,0,0,0
2021-10-19 07:04:00,0,0,0,0,0,0,0,0
2021-04-17 10:46:00,0,0,0,0,0,0,0,0
2021-03-07 19:50:00,0,0,0,0,0,0,0,0


### Métricas generales:

In [19]:
metodos_SH_lista = ['HBOS','IForest','KNN']
metodos_SH_metricas = calc_metricas(metodos_SH,metodos_SH_lista)
metodos_SH_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
HBOS,0.605282,0.733614,0.848297,0.25861,0.230769,0.979796
IForest,0.606904,0.731625,0.803317,0.260117,0.242926,0.970881
KNN,0.539609,0.672711,0.50704,0.096504,0.151175,0.928043


In [20]:
metodos_PvP_metricas = calc_metricas(metodos_PvP, metodos_PvP_lista)
metodos_PvP_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
MAD_PyOD,0.598938,0.72659,0.795289,0.242067,0.226408,0.971468
MAD_PySAD,0.601583,0.726984,0.780426,0.247209,0.235621,0.967545
SAD,0.601446,0.726913,0.78047,0.246909,0.235294,0.967598


In [21]:
px.line(metodos_SH_metricas.T)

### Sensibilidad por tipo de anomalía

In [22]:
metodos_SH_sensibilidades = calc_sensibilidad(metodos_SH,metodos_SH_lista,anomalias)
metodos_SH_sensibilidades

,Anomalia,Negro,Congelado,Hum,CurvaCola
HBOS,0.230769,0.84748,0.907783,0.494983,0.114838
IForest,0.242926,0.820264,0.908347,0.494983,0.129249
KNN,0.151175,0.270816,0.286802,0.505017,0.130671


In [23]:
px.line(metodos_SH_sensibilidades.T)

## Resumen Final

In [24]:
resumen_general = pd.concat([metodos_metricas.T.add_suffix('_general'),metodos_NvS_metricas.T.add_suffix('_NvS'),
            metodos_PvP_metricas.T.add_suffix('_PvP'),metodos_SH_metricas.T.add_suffix('_SH')],axis=1).T

px.line(resumen_general.T)

In [25]:
resumen_general_sensibilidades = pd.concat([metodos_sensibilidades.T.add_suffix('_general'),metodos_NvS_sensibilidades.T.add_suffix('_NvS'),
            metodos_PvP_sensibilidades.T.add_suffix('_PvP'),metodos_SH_sensibilidades.T.add_suffix('_SH')],axis=1).T

px.line(resumen_general_sensibilidades.T)

# Resumen Anomalías Thresholds

In [26]:
anomalias = ['Anomalia','Negro','Congelado','Hum']
metricas = ['roc_auc','accuracy','precision','kappa','sensibilidad','especificidad']

## Métodos

In [27]:
metodos = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PyOD/Resultados/HBOS_completo.csv',index_col='fecha')
metodos = metodos.drop('Y_pred', axis=1)

metodos['HBOS'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PyOD/Resultados/HBOS_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['IForest'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PyOD/Resultados/IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['KNN'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PyOD/Resultados/KNN_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['LODA'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PyOD/Resultados/LODA_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MAD_PyOD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PyOD/Resultados/MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MCD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PyOD/Resultados/MCD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos['IForestASD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PySAD/Resultados/IForestASD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MAD_PySAD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PySAD/Resultados/MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['SAD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PySAD/Resultados/SAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['xStream'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/PySAD/Resultados/xStream_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_htm = pd.read_csv('Metodos_TipoAnomalia/Metodos_Thresholds/HTMs/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_htm['HTM'] = metodos_htm['Y_pred']
metodos_htm = metodos_htm.drop('Y_pred',axis=1)

metodos.head()

,Negro,Congelado,Hum,Anomalia,HBOS,IForest,KNN,LODA,MAD_PyOD,MCD,IForestASD,MAD_PySAD,SAD,xStream
fecha,,,,,,,,,,,,,,
2021-05-05 10:31:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-03-04 02:06:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-07-06 21:41:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-05-01 21:58:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-04-03 19:13:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Métricas generales:

In [28]:
metodos_lista = ['HBOS','IForest','KNN','LODA','MAD_PyOD','MCD', 'IForestASD', 'MAD_PySAD','SAD','xStream']
metodos_metricas = calc_metricas(metodos,metodos_lista)
metodos_metricas = pd.concat([metodos_metricas,calc_metricas(metodos_htm, ['HTM'])])
metodos_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
HBOS,0.61892,0.875366,0.227216,0.19994,0.320423,0.917418
IForest,0.72485,0.902897,0.366127,0.377521,0.517606,0.932093
KNN,0.547181,0.856342,0.13247,0.079206,0.187324,0.907039
LODA,0.585794,0.866114,0.182788,0.143714,0.259507,0.912082
MAD_PyOD,0.948249,0.98244,0.852048,0.869887,0.908451,0.988046
MCD,0.549841,0.856144,0.135288,0.083194,0.19331,0.906372
IForestASD,0.8418,0.789077,0.237632,0.297964,0.903169,0.780431
MAD_PySAD,0.945623,0.977256,0.796851,0.836911,0.908803,0.982443
SAD,0.934854,0.957538,0.639881,0.728425,0.908451,0.961257
xStream,0.879511,0.844362,0.301743,0.389743,0.920423,0.838599


In [29]:
px.line(metodos_metricas.T)

### Sensibilidad por tipo de anomalía

In [30]:
metodos_sensibilidades = calc_sensibilidad(metodos,metodos_lista,anomalias)
metodos_sensibilidades = pd.concat([metodos_sensibilidades,calc_sensibilidad(metodos_htm, ['HTM'], anomalias)])
metodos_sensibilidades

,Anomalia,Negro,Congelado,Hum
HBOS,0.320423,0.272241,0.259173,0.890909
IForest,0.517606,0.508508,0.486339,0.807273
KNN,0.187324,0.122022,0.162763,0.407273
LODA,0.259507,0.187652,0.188525,0.912727
MAD_PyOD,0.908451,1.0,0.898517,1.0
MCD,0.19331,0.111327,0.161202,0.483636
IForestASD,0.903169,0.944579,0.895004,0.981818
MAD_PySAD,0.908803,1.0,0.898907,1.0
SAD,0.908451,1.0,0.898517,1.0
xStream,0.920423,0.98104,0.922717,0.898182


In [31]:
px.line(metodos_sensibilidades.T)

## Métodos NIRvsSecadero

In [32]:
metodos_NvS = pd.read_csv('Metodos_NIRvsSecadero/Thresholds/Resultados/PyOD_IForest_completo.csv',index_col='fecha')
metodos_NvS = metodos_NvS.drop('Y_pred', axis=1)

metodos_NvS['IForest'] = pd.read_csv('Metodos_NIRvsSecadero/Thresholds/Resultados/PyOD_IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_NvS['MAD'] = pd.read_csv('Metodos_NIRvsSecadero/Thresholds/Resultados/PyOD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_NvS_htm = pd.read_csv('Metodos_NIRvsSecadero/Thresholds/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_NvS_htm['HTM'] = metodos_NvS_htm['Y_pred']
metodos_NvS_htm = metodos_NvS_htm.drop('Y_pred',axis=1)

metodos_NvS.head()

,Negro,Congelado,Hum,Anomalia,IForest,MAD
fecha,,,,,,
2021-05-05 10:31:00,0,0,0,0,0,0
2021-03-04 02:06:00,0,0,0,0,0,0
2021-07-06 21:41:00,0,0,0,0,0,0
2021-05-01 21:58:00,0,0,0,0,0,0
2021-04-03 19:13:00,0,0,0,0,0,0


### Métricas generales:

In [33]:
metodos_NvS_lista = ['IForest','MAD']
metodos_NvS_metricas = calc_metricas(metodos_NvS,['IForest','MAD'])
metodos_NvS_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
IForest,0.88955,0.943797,0.56966,0.644804,0.826408,0.952692
MAD,0.88998,0.947319,0.590404,0.659731,0.823239,0.956721


In [34]:
metodos_NvS_metricas = calc_metricas(metodos_NvS, metodos_NvS_lista)
metodos_NvS_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
IForest,0.88955,0.943797,0.56966,0.644804,0.826408,0.952692
MAD,0.88998,0.947319,0.590404,0.659731,0.823239,0.956721


In [35]:
px.line(metodos_NvS_metricas.T)

### Sensibilidad por tipo de anomalía

In [36]:
metodos_NvS_sensibilidades = calc_sensibilidad(metodos_NvS,metodos_NvS_lista,anomalias)
metodos_NvS_sensibilidades

,Anomalia,Negro,Congelado,Hum
IForest,0.826408,1.0,0.90242,0.116364
MAD,0.823239,0.999514,0.901639,0.090909


In [37]:
px.line(metodos_NvS_sensibilidades.T)

## Métodos PVvsPred

In [38]:
metodos_PvP = pd.read_csv('Metodos_PVvsPred/Thresholds/Resultados/PyOD_MAD_completo.csv',index_col='fecha')
metodos_PvP = metodos_PvP.drop('Y_pred', axis=1)

metodos_PvP['MAD_PyOD'] = pd.read_csv('Metodos_PVvsPred/Thresholds/Resultados/PyOD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_PvP['MAD_PySAD'] = pd.read_csv('Metodos_PVvsPred/Thresholds/Resultados/PySAD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_PvP['SAD'] = pd.read_csv('Metodos_PVvsPred/Thresholds/Resultados/PySAD_SAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_PvP_htm = pd.read_csv('Metodos_PVvsPred/Thresholds/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_PvP_htm['HTM'] = metodos_PvP_htm['Y_pred']
metodos_PvP_htm = metodos_PvP_htm.drop('Y_pred',axis=1)

metodos_PvP.head()

,Negro,Congelado,Hum,Anomalia,MAD_PyOD,MAD_PySAD,SAD
fecha,,,,,,,
2021-05-05 10:31:00,0,0,0,0,0,0,0
2021-03-04 02:06:00,0,0,0,0,0,0,0
2021-07-06 21:41:00,0,0,0,0,0,0,0
2021-05-01 21:58:00,0,0,0,0,0,0,0
2021-04-03 19:13:00,0,0,0,0,0,0,0


### Métricas generales:

In [39]:
metodos_PvP_lista = ['MAD_PyOD','MAD_PySAD','SAD']
metodos_PvP_metricas = calc_metricas(metodos_PvP,metodos_PvP_lista)
metodos_PvP_metricas = pd.concat([metodos_PvP_metricas,calc_metricas(metodos_PvP_htm, ['HTM'])])
metodos_PvP_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
MAD_PyOD,0.900035,0.937274,0.534138,0.625516,0.85669,0.94338
MAD_PySAD,0.903856,0.946798,0.583634,0.665348,0.853873,0.95384
SAD,0.902974,0.946972,0.584865,0.665588,0.851761,0.954186
HTM,0.456342,0.83696,0.010094,-0.075064,0.013657,0.899028


In [40]:
metodos_PvP_metricas = calc_metricas(metodos_PvP, metodos_PvP_lista)
metodos_PvP_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
MAD_PyOD,0.900035,0.937274,0.534138,0.625516,0.85669,0.94338
MAD_PySAD,0.903856,0.946798,0.583634,0.665348,0.853873,0.95384
SAD,0.902974,0.946972,0.584865,0.665588,0.851761,0.954186


In [41]:
px.line(metodos_PvP_metricas.T)

### Sensibilidad por tipo de anomalía

In [42]:
metodos_PvP_sensibilidades = calc_sensibilidad(metodos_PvP,metodos_PvP_lista,anomalias)
metodos_PvP_sensibilidades = pd.concat([metodos_PvP_sensibilidades,calc_sensibilidad(metodos_PvP_htm, ['HTM'], anomalias)])
metodos_PvP_sensibilidades

,Anomalia,Negro,Congelado,Hum
MAD_PyOD,0.85669,1.0,0.901639,0.436364
MAD_PySAD,0.853873,1.0,0.900859,0.414545
SAD,0.851761,1.0,0.900078,0.4
HTM,0.013657,0.0,0.013573,0.014573


In [43]:
px.line(metodos_PvP_sensibilidades.T)

## Métodos SoloHum

In [44]:
metodos_SH = pd.read_csv('Metodos_SoloHumedades/Thresholds/Resultados/PyOD_HBOS_completo.csv',index_col='fecha')
metodos_SH = metodos_SH.drop('Y_pred', axis=1)

metodos_SH['HBOS'] = pd.read_csv('Metodos_SoloHumedades/Thresholds/Resultados/PyOD_HBOS_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_SH['IForest'] = pd.read_csv('Metodos_SoloHumedades/Thresholds/Resultados/PyOD_IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_SH['KNN'] = pd.read_csv('Metodos_SoloHumedades/Thresholds/Resultados/PyOD_KNN_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_SH.head()

,Negro,Congelado,Hum,Anomalia,HBOS,IForest,KNN
fecha,,,,,,,
2021-05-05 10:31:00,0,0,0,0,0,0,0
2021-03-04 02:06:00,0,0,0,0,0,0,0
2021-07-06 21:41:00,0,0,0,0,0,0,0
2021-05-01 21:58:00,0,0,0,0,0,0,0
2021-04-03 19:13:00,0,0,0,0,0,0,0


### Métricas generales:

In [45]:
metodos_SH_lista = ['HBOS','IForest','KNN']
metodos_SH_metricas = calc_metricas(metodos_SH,metodos_SH_lista)
metodos_SH_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
HBOS,0.915374,0.961556,0.678968,0.738893,0.86162,0.969129
IForest,0.907066,0.948832,0.594779,0.675707,0.858451,0.955681
KNN,0.597926,0.868099,0.196918,0.162794,0.283451,0.912402


In [46]:
metodos_PvP_metricas = calc_metricas(metodos_PvP, metodos_PvP_lista)
metodos_PvP_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
MAD_PyOD,0.900035,0.937274,0.534138,0.625516,0.85669,0.94338
MAD_PySAD,0.903856,0.946798,0.583634,0.665348,0.853873,0.95384
SAD,0.902974,0.946972,0.584865,0.665588,0.851761,0.954186


In [47]:
px.line(metodos_SH_metricas.T)

### Sensibilidad por tipo de anomalía

In [48]:
metodos_SH_sensibilidades = calc_sensibilidad(metodos_SH,metodos_SH_lista,anomalias)
metodos_SH_sensibilidades

,Anomalia,Negro,Congelado,Hum
HBOS,0.86162,1.0,0.900859,0.494545
IForest,0.858451,1.0,0.900078,0.469091
KNN,0.283451,0.258629,0.266979,0.429091


In [49]:
px.line(metodos_SH_sensibilidades.T)

## Resumen Final

In [50]:
resumen_thresholds = pd.concat([metodos_metricas.T.add_suffix('_general'),metodos_NvS_metricas.T.add_suffix('_NvS'),
            metodos_PvP_metricas.T.add_suffix('_PvP'),metodos_SH_metricas.T.add_suffix('_SH')],axis=1).T

px.line(resumen_thresholds.T)

In [51]:
resumen_thresholds_sensibilidades = pd.concat([metodos_sensibilidades.T.add_suffix('_general'),metodos_NvS_sensibilidades.T.add_suffix('_NvS'),
            metodos_PvP_sensibilidades.T.add_suffix('_PvP'),metodos_SH_sensibilidades.T.add_suffix('_SH')],axis=1).T

px.line(resumen_thresholds_sensibilidades.T)

# Resumen Anomalía CurvaCola

In [52]:
anomalias = ['CurvaCola']
metricas = ['roc_auc','accuracy','precision','kappa','sensibilidad','especificidad']

def calc_metricas(metodos,metodos_lista):
    metodos_metricas = pd.DataFrame(index=metodos_lista,columns=metricas)
    for metodo in metodos_lista:
        metodos_metricas.loc[metodo,metricas[0]] = roc_auc_score(metodos['CurvaCola'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[1]] = accuracy_score(metodos['CurvaCola'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[2]] = precision_score(metodos['CurvaCola'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[3]] = cohen_kappa_score(metodos['CurvaCola'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[4]] = recall_score(metodos['CurvaCola'], metodos[metodo])
        metodos_metricas.loc[metodo,metricas[5]] = recall_score(metodos['CurvaCola'], metodos[metodo], pos_label=0)
    return metodos_metricas

def calc_sensibilidad(metodos,metodos_lista,anomalias):
    metodos_sensibilidades = pd.DataFrame(index=metodos_lista, columns=anomalias)
    for metodo in metodos_lista:
        for anomalia in anomalias:
            metodos_sensibilidades.loc[metodo,anomalia] = recall_score(metodos[anomalia], metodos[metodo])
    return metodos_sensibilidades

## Métodos

In [53]:
metodos = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PyOD/Resultados/HBOS_completo.csv',index_col='fecha')
metodos = metodos.drop('Y_pred', axis=1)

metodos['HBOS'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PyOD/Resultados/HBOS_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['IForest'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PyOD/Resultados/IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['KNN'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PyOD/Resultados/KNN_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['LODA'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PyOD/Resultados/LODA_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MAD_PyOD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PyOD/Resultados/MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MCD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PyOD/Resultados/MCD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos['IForestASD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PySAD/Resultados/IForestASD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['MAD_PySAD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PySAD/Resultados/MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['SAD'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PySAD/Resultados/SAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos['xStream'] = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/PySAD/Resultados/xStream_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_htm = pd.read_csv('Metodos_TipoAnomalia/Metodos_CurvaCola/HTMs/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_htm['HTM'] = metodos_htm['Y_pred']
metodos_htm = metodos_htm.drop('Y_pred',axis=1)

metodos.head()

,CurvaCola,HBOS,IForest,KNN,LODA,MAD_PyOD,MCD,IForestASD,MAD_PySAD,SAD,xStream
fecha,,,,,,,,,,,
2021-11-26 11:48:00,1,0,0,0,0,0,0,0,0,0,0
2021-06-06 19:17:00,0,0,0,0,0,1,0,0,1,1,0
2021-12-04 07:59:00,1,0,0,0,0,0,0,0,0,0,0
2021-11-06 17:55:00,1,0,0,0,0,0,0,0,0,0,0
2021-05-05 08:19:00,0,0,0,1,0,0,0,0,0,0,1


### Métricas generales:

In [54]:
metodos_lista = ['HBOS','IForest','KNN','LODA','MAD_PyOD','MCD', 'IForestASD', 'MAD_PySAD','SAD','xStream']
metodos_metricas = calc_metricas(metodos,metodos_lista)
metodos_metricas = pd.concat([metodos_metricas,calc_metricas(metodos_htm, ['HTM'])])
metodos_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
HBOS,0.50621,0.689658,0.291055,0.015438,0.111893,0.900527
IForest,0.5111,0.694128,0.310265,0.02765,0.117686,0.904515
KNN,0.50931,0.693893,0.304262,0.023277,0.112553,0.906067
LODA,0.507536,0.690952,0.296261,0.018748,0.113286,0.901785
MAD_PyOD,0.496859,0.724635,0.163636,-0.009019,0.007259,0.986459
MCD,0.516269,0.698461,0.330611,0.040561,0.124652,0.907887
IForestASD,0.511881,0.647309,0.290273,0.025716,0.22078,0.802981
MAD_PySAD,0.495933,0.720106,0.187867,-0.011516,0.014078,0.977788
SAD,0.495873,0.720086,0.186457,-0.011687,0.013932,0.977815
xStream,0.513001,0.63882,0.290175,0.027432,0.242558,0.783445


In [55]:
px.line(metodos_metricas.T)

## Métodos NIRvsSecadero

In [56]:
metodos_NvS = pd.read_csv('Metodos_NIRvsSecadero/CurvaCola/Resultados/PyOD_IForest_completo.csv',index_col='fecha')
metodos_NvS = metodos_NvS.drop('Y_pred', axis=1)

metodos_NvS['IForest'] = pd.read_csv('Metodos_NIRvsSecadero/CurvaCola/Resultados/PyOD_IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_NvS['MAD'] = pd.read_csv('Metodos_NIRvsSecadero/CurvaCola/Resultados/PyOD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_NvS_htm = pd.read_csv('Metodos_NIRvsSecadero/CurvaCola/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_NvS_htm['HTM'] = metodos_NvS_htm['Y_pred']
metodos_NvS_htm = metodos_NvS_htm.drop('Y_pred',axis=1)

metodos_NvS.head()

,CurvaCola,IForest,MAD
fecha,,,
2021-11-26 11:48:00,1,0,0
2021-06-06 19:17:00,0,1,0
2021-12-04 07:59:00,1,0,0
2021-11-06 17:55:00,1,0,0
2021-05-05 08:19:00,0,0,0


### Métricas generales:

In [57]:
metodos_NvS_lista = ['IForest','MAD']
metodos_NvS_metricas = calc_metricas(metodos_NvS,['IForest','MAD'])
metodos_NvS_metricas = pd.concat([metodos_NvS_metricas,calc_metricas(metodos_NvS_htm, ['HTM'])])
metodos_NvS_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
IForest,0.495745,0.684011,0.250252,-0.010665,0.091069,0.90042
MAD,0.504158,0.72687,0.35157,0.011788,0.025444,0.982873
HTM,0.456544,0.666068,0.03846,-0.112812,0.010848,0.902239


In [58]:
px.line(metodos_NvS_metricas.T)

## Métodos PVvsPred

In [59]:
metodos_PvP = pd.read_csv('Metodos_PVvsPred/CurvaCola/Resultados/PyOD_MAD_completo.csv',index_col='fecha')
metodos_PvP = metodos_PvP.drop('Y_pred', axis=1)

metodos_PvP['MAD_PyOD'] = pd.read_csv('Metodos_PVvsPred/CurvaCola/Resultados/PyOD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_PvP['MAD_PySAD'] = pd.read_csv('Metodos_PVvsPred/CurvaCola/Resultados/PySAD_MAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_PvP['SAD'] = pd.read_csv('Metodos_PVvsPred/CurvaCola/Resultados/PySAD_SAD_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')

metodos_PvP_htm = pd.read_csv('Metodos_PVvsPred/CurvaCola/HTM_Uni/HTM_completo.csv',index_col='fecha')
metodos_PvP_htm['HTM'] = metodos_PvP_htm['Y_pred']
metodos_PvP_htm = metodos_PvP_htm.drop('Y_pred',axis=1)

metodos_PvP.head()

,CurvaCola,MAD_PyOD,MAD_PySAD,SAD
fecha,,,,
2021-11-26 11:48:00,1,0,0,0
2021-06-06 19:17:00,0,0,0,0
2021-12-04 07:59:00,1,0,0,0
2021-11-06 17:55:00,1,0,0,0
2021-05-05 08:19:00,0,0,0,0


### Métricas generales:

In [60]:
metodos_PvP_lista = ['MAD_PyOD','MAD_PySAD','SAD']
metodos_PvP_metricas = calc_metricas(metodos_PvP,metodos_PvP_lista)
metodos_PvP_metricas = pd.concat([metodos_PvP_metricas,calc_metricas(metodos_PvP_htm, ['HTM'])])
metodos_PvP_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
MAD_PyOD,0.517988,0.727144,0.434965,0.04912,0.068412,0.967565
MAD_PySAD,0.522436,0.702755,0.353586,0.05584,0.134844,0.910028
SAD,0.532131,0.710479,0.39117,0.080003,0.148775,0.915487
HTM,0.489879,0.71982,0.012792,-0.028955,0.000755,0.979003


In [61]:
px.line(metodos_PvP_metricas.T)

## Métodos SoloHum

In [62]:
metodos_SH = pd.read_csv('Metodos_SoloHumedades/CurvaCola/Resultados/PyOD_HBOS_completo.csv',index_col='fecha')
metodos_SH = metodos_SH.drop('Y_pred', axis=1)

metodos_SH['HBOS'] = pd.read_csv('Metodos_SoloHumedades/CurvaCola/Resultados/PyOD_HBOS_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_SH['IForest'] = pd.read_csv('Metodos_SoloHumedades/CurvaCola/Resultados/PyOD_IForest_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_SH['KNN'] = pd.read_csv('Metodos_SoloHumedades/CurvaCola/Resultados/PyOD_KNN_completo.csv',usecols=['fecha','Y_pred'],index_col='fecha')
metodos_SH.head()

,CurvaCola,HBOS,IForest,KNN
fecha,,,,
2021-11-26 11:48:00,1,0,0,0
2021-06-06 19:17:00,0,1,0,0
2021-12-04 07:59:00,1,0,0,0
2021-11-06 17:55:00,1,0,0,0
2021-05-05 08:19:00,0,0,0,0


### Métricas generales:

In [63]:
metodos_SH_lista = ['HBOS','IForest','KNN']
metodos_SH_metricas = calc_metricas(metodos_SH,metodos_SH_lista)
metodos_SH_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
HBOS,0.507856,0.694697,0.299876,0.019765,0.106247,0.909466
IForest,0.517886,0.699363,0.336357,0.04454,0.127805,0.907967
KNN,0.513567,0.696343,0.320109,0.033824,0.120692,0.906442


In [64]:
metodos_PvP_metricas = calc_metricas(metodos_PvP, metodos_PvP_lista)
metodos_PvP_metricas

,roc_auc,accuracy,precision,kappa,sensibilidad,especificidad
MAD_PyOD,0.517988,0.727144,0.434965,0.04912,0.068412,0.967565
MAD_PySAD,0.522436,0.702755,0.353586,0.05584,0.134844,0.910028
SAD,0.532131,0.710479,0.39117,0.080003,0.148775,0.915487


In [65]:
px.line(metodos_SH_metricas.T)

## Resumen Final

In [66]:
resumen_curvacola = pd.concat([metodos_metricas.T.add_suffix('_general'),metodos_NvS_metricas.T.add_suffix('_NvS'),
            metodos_PvP_metricas.T.add_suffix('_PvP'),metodos_SH_metricas.T.add_suffix('_SH')],axis=1).T

px.line(resumen_curvacola.T)

# Total

In [67]:
total = pd.concat([resumen_general.T.add_suffix('_general'),resumen_thresholds.T.add_suffix('_thresholds'),resumen_curvacola.T.add_suffix('_curvacola')],axis=1).T
px.line(total.T)